Used to measure how long running REPL code would take over a full training session.

Running the code linearly resulted in ~12 seconds of running code which seemed fine.

In [1]:
import pdb
import os
import torch
import pickle
import datetime
from tqdm import tqdm

from _ppo import PPO
from _seq2seq import Seq2Seq
from _memory import Memory
from _repl_env import ReplEnv
from _language import InputLang, OutputLang

from _constants import (
    DEVICE,
    ACTION_DIM,
    LOG_INTERVAL,
    MAX_EPISODESS,
    MAX_EPISODES_TIMESTEPS,
    UPDATE_EVERY_N_EPISODESS,
    MIN_TRAINING_SAMPLE_REWARD,
    LEARNING_RATE,
    BETAS,
    GAMMA,
    K_EPOCHS,
    EPS_CLIP,
    SEQ2SEQ_LOAD_FROM,
    SAVE_EVERY_N_EPISODESS,
    RENDER_EVERY_N_EPISODESS,
    RENDER,
    LOG_SERIES_EVERY_N_EPISODESS,
    LOGS_FOLDER,
    RENDER_FILE,
    MODELS_FOLDER,
    SEQ2SEQ_PRE_TRAIN_STEPS,
    SEQ2SEQ_BATCH_SIZE,
    SEQ2SEQ_PRE_TRAINED,
)

In [2]:
def _run_code(code):
    try:
        output = str(eval(code, None, None))
    except Exception:
        output = 'E'
    return output[:OutputLang.maxlen]

In [17]:
PPO_MAX_BATCH_SIZE = 100
N_EPISODES = 1000

In [18]:
def rand_code_and_output():
    code_batches = torch.randint(
        0, len(InputLang.chars),
        (N_EPISODES, PPO_MAX_BATCH_SIZE, InputLang.maxlen)
    )
    code_batches[:,:,-1] = InputLang.chars.index('EOS')

    output_batches = torch.zeros(
        (N_EPISODES, PPO_MAX_BATCH_SIZE, OutputLang.maxlen)
    )
    
    return code_batches, output_batches

In [19]:
from tqdm import tqdm
from time import time

### Linear

In [20]:
code_batches, output_batches = rand_code_and_output()

In [21]:
start = time()

batch_i = 0
for batch in code_batches:
    step_i = 0
    for step in batch:
        output_batches[batch_i, step_i] =  OutputLang.str_to_padded_tensor(_run_code(
            InputLang.tensor_to_str(step)
        ))[0]
        step_i += 1
    batch_i += 1

print(int(10* (time() - start) ), 'm-seconds')

122 m-seconds


### Multi-Process

In [6]:
code_batches, output_batches = rand_code_and_output()

In [12]:
start = time()

from torch import multiprocessing

BATCH_COUNT = TEST_TRAIN_STEPS // PPO_MAX_BATCH_SIZE


def get_batch_outputs(batch_i, return_dict, code_batch):
    output_batch = torch.zeros(PPO_MAX_BATCH_SIZE, OutputLang.maxlen)
    for step_i in range(PPO_MAX_BATCH_SIZE):
        output_batches[step_i] =  OutputLang.str_to_padded_tensor(_run_code(
            InputLang.tensor_to_str(code_batch[step_i])
        ))[0]
    return_dict[batch_i] = output_batch

In [13]:
BATCH_COUNT

1000

In [14]:
# uses max process count
pool = multiprocessing.Pool()
pool.close()

In [20]:
def f(a, i, b):
    a[i] = b

z = torch.zeros(10)

pool = multiprocessing.Pool()
z = pool.map(f, torch.ones(10))
pool.close()

In [15]:
help(pool.map)

Help on method map in module multiprocessing.pool:

map(func, iterable, chunksize=None) method of multiprocessing.pool.Pool instance
    Apply `func` to each element in `iterable`, collecting the results
    in a list that is returned.



In [72]:
batch_i_with_batch_code.size()

torch.Size([1000, 701])

In [74]:
get_batch_outputs(batch_i_with_batch_code[0])

IndexError: dimension specified as 0 but tensor has no dimensions

In [79]:
start = time()

from torch import multiprocessing

BATCH_COUNT = TEST_TRAIN_STEPS // PPO_MAX_BATCH_SIZE


def get_batch_outputs(batch_i_with_batch_code_and_output_holder):
    batch_i = batch_i_with_batch_code[0]
    code_end = PPO_MAX_BATCH_SIZE * InputLang.maxlen
    code_batch = batch_i_with_batch_code[1:code_end].view(PPO_MAX_BATCH_SIZE, -1)
    output_batch = batch_i_with_batch_code[code_end: PPO_MAX_BATCH_SIZE * OutputLang.maxlen].view(PPO_MAX_BATCH_SIZE, -1)
    for step_i in range(PPO_MAX_BATCH_SIZE):
        output_batches[step_i] =  OutputLang.str_to_padded_tensor(_run_code(
            InputLang.tensor_to_str(code_batch[step_i])
        ))[0]


if __name__ == '__main__':
    batch_i_with_batch_code_and_output_holder = torch.cat(
        (
            torch.arange(BATCH_COUNT).view(BATCH_COUNT, 1),
            code_batches.view(BATCH_COUNT, -1),
            torch.zeros((BATCH_COUNT, PPO_MAX_BATCH_SIZE, OutputLang.maxlen), dtype=torch.long).view(BATCH_COUNT, -1)
        ),
        dim=1
    )    
    
    pool = multiprocessing.Pool()
    pool.map(get_batch_outputs, batch_i_with_batch_code_and_output_holder)
    pool.close()
    output_batch = batch_i_with_batch_code[PPO_MAX_BATCH_SIZE * InputLang.maxlen: PPO_MAX_BATCH_SIZE * OutputLang.maxlen].view(PPO_MAX_BATCH_SIZE, -1)
    print(output_batch)

RuntimeError: shape '[100, -1]' is invalid for input of size 489999

In [9]:
start = time()

import multiprocessing

BATCH_COUNT = TEST_TRAIN_STEPS // PPO_MAX_BATCH_SIZE


def get_batch_outputs(batch_i, return_dict, code_batch):
    output_batch = torch.zeros(PPO_MAX_BATCH_SIZE, OutputLang.maxlen)
    for step_i in range(PPO_MAX_BATCH_SIZE):
        output_batches[step_i] =  OutputLang.str_to_padded_tensor(_run_code(
            InputLang.tensor_to_str(code_batch[step_i])
        ))[0]
    return_dict[batch_i] = output_batch


if __name__ == '__main__':
    manager = multiprocessing.Manager()
    return_dict = manager.dict()
    
    jobs = [multiprocessing.Process(target=get_batch_outputs, args=(batch_i, return_dict, code_batches[batch_i])) for batch_i in range(BATCH_COUNT)]
    for proc in jobs:
        proc.start()
    for proc in jobs:
        proc.join()
        proc.close()
    
    for i in range(BATCH_COUNT):
        output_batches[i] = return_dict[i]

Process Process-174:
Traceback (most recent call last):
  File "/Users/Fraser/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/Fraser/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-9-77e9dca9f873>", line 14, in get_batch_outputs
    return_dict[batch_i] = output_batch
  File "<string>", line 2, in __setitem__
  File "/Users/Fraser/anaconda3/lib/python3.7/multiprocessing/managers.py", line 834, in _callmethod
    raise convert_to_error(kind, result)
multiprocessing.managers.RemoteError: 
---------------------------------------------------------------------------
Traceback (most recent call last):
  File "/Users/Fraser/anaconda3/lib/python3.7/multiprocessing/managers.py", line 234, in serve_client
    request = recv()
  File "/Users/Fraser/anaconda3/lib/python3.7/multiprocessing/connection.py", line 251, in recv
    return _ForkingPickler.lo

OSError: [Errno 24] Too many open files

In [22]:
torch.bool

torch.bool

In [24]:
r = torch.randn(10)
r

tensor([-0.8465,  0.4302,  0.0279, -0.4834, -0.1913, -0.1155,  0.2675, -0.2440,
         0.5253,  0.5512])

In [27]:
r[1:4] = torch.zeros(3)
r

tensor([-0.8465,  0.0000,  0.0000,  0.0000, -0.1913, -0.1155,  0.2675, -0.2440,
         0.5253,  0.5512])